In [1]:
import pandas as pd
import glob
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_percentage_error

In [ ]:
os.chdir('/home1/gkrtod35/Diffusion-TS/Data')

In [3]:
df = pd.read_csv('merged_data_processed_seoul.csv', low_memory=False)
df.head()

,Idx,date,time,solar generation,일시,기온(°C),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),...,일조(hr),일사(MJ/m2),적설(cm),전운량(10분위),중하층운량(10분위),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,0,2014-01-01,0,0.0,2014-01-01 00:00,3.3,3.8,250.0,65.0,5.0,...,0.0,0.0,0.0,6.0,6.0,0.0,0.1,-0.2,0.0,1.5
1,0,2014-01-01,1,0.0,2014-01-01 01:00,2.6,2.3,250.0,66.0,4.9,...,0.0,0.0,0.0,0.0,0.0,-0.1,0.1,-0.2,0.1,1.5
2,0,2014-01-01,2,0.0,2014-01-01 02:00,1.7,1.7,250.0,67.0,4.6,...,0.0,0.0,0.0,0.0,0.0,-0.3,0.0,-0.2,0.0,1.5
3,0,2014-01-01,3,0.0,2014-01-01 03:00,1.4,1.4,250.0,60.0,4.1,...,0.0,0.0,0.0,0.0,0.0,-0.4,0.0,-0.2,0.1,1.5
4,0,2014-01-01,4,0.0,2014-01-01 04:00,0.9,2.8,270.0,59.0,3.8,...,0.0,0.0,0.0,0.0,0.0,-0.6,0.0,-0.2,0.0,1.5


In [4]:
# 
seq_len  = 8760   # 학습용: 1년치 (1h 단위 → 8760h)
pred_len =   24   # 테스트용: 1일치 (24h)

# 꼬리(tail)에서 잘라내기
#    – train_data: 마지막(pred_len)시간 바로 앞의 seq_len시간
#    – test_data : 마지막 pred_len시간
df = df[-(seq_len + pred_len) : ]  # shape (8760, C)

In [5]:
numeric_df = df.drop(columns=['Idx','date','time','일시'])
numeric_df = numeric_df.apply(pd.to_numeric, errors='coerce')

In [4]:
class SlidingWindowDataset(Dataset):
    def __init__(self, data, seq_len=720, pred_len=24, stride=24):
        """
        data: array-like [T, C]
        seq_len: 입력 길이 (예: 720)
        pred_len: 예측 길이 (예: 24)
        stride: 윈도우 이동 간격 (예: 1)
        """
        import numpy as np
        data = np.asarray(data, dtype=np.float32)
        T, C  = data.shape
        self.X, self.Y = [], []
        for i in range(0, T - seq_len - pred_len + 1, stride):
            self.X.append(data[i : i+seq_len])                          # [seq_len, C]
            self.Y.append(data[i+seq_len : i+seq_len+pred_len, 0])      # [pred_len]
        self.X = torch.from_numpy(np.stack(self.X))  # [N, seq_len, C]
        self.Y = torch.from_numpy(np.stack(self.Y))  # [N, pred_len]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # [C, seq_len], [pred_len]
        return self.X[idx].transpose(0,1), self.Y[idx]

In [5]:
# 데이터셋 정의: sliding window로 (X, y) 생성
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_len=8760, pred_len=24):
        """
        data: NumPy array of shape [T, C] (시간 T, 채널 C)
        seq_len: 입력 시퀀스 길이 (8760)
        pred_len: 예측할 시퀀스 길이 (24)
        """
        data = np.asarray(data, dtype=np.float32)
        
        self.X = []
        self.Y = []
        T, C = data.shape
        for i in range(T - seq_len - pred_len + 1):
            self.X.append(data[i:i+seq_len])
            self.Y.append(data[i+seq_len:i+seq_len+pred_len, 0])  # 채널0=solar 예측
        self.X = torch.tensor(self.X, dtype=torch.float32)      # [N, seq_len, C]
        self.Y = torch.tensor(self.Y, dtype=torch.float32)      # [N, pred_len]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # 모델에 넣기 편하게 채널 차원을 앞쪽으로 옮김: [C, seq_len]
        return self.X[idx].transpose(0,1), self.Y[idx]

In [6]:
# 모델 정의: CNN → LSTM → FC
class CNN_LSTM(nn.Module):
    def __init__(self, in_channels, cnn_channels=32, lstm_hidden=64, pred_len=24):
        super().__init__()
        # 1D-CNN: 채널별 시계열 특성 추출
        self.conv1 = nn.Conv1d(in_channels, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, cnn_channels, kernel_size=3, padding=1)
        self.relu  = nn.ReLU()
        self.pool  = nn.MaxPool1d(2)  # 길이 절반
        
        # LSTM: 시계열 의존성 학습
        self.lstm = nn.LSTM(
            input_size=cnn_channels,
            hidden_size=lstm_hidden,
            num_layers=2,
            batch_first=True
        )
        # 최종 24시간 예측용 FC
        self.fc = nn.Linear(lstm_hidden, pred_len)

    def forward(self, x):
        # x: [B, C, seq_len]
        x = self.relu(self.conv1(x))     # [B, 16, L]
        x = self.relu(self.conv2(x))     # [B, cnn_channels, L]
        x = self.pool(x)                 # [B, cnn_channels, L/2]
        
        # LSTM 입력: (batch, time, feat)
        x = x.transpose(1,2)             # [B, L/2, cnn_channels]
        out, _ = self.lstm(x)            # out: [B, L/2, lstm_hidden]
        
        # 마지막 시점만 사용해 예측
        out = out[:, -1, :]              # [B, lstm_hidden]
        y_hat = self.fc(out)             # [B, pred_len]
        return y_hat

In [7]:
# 학습/평가 루프
def train(model, loader, optim, criterion, device):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optim.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optim.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(loader.dataset)

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            total_loss += criterion(y_pred, y).item() * x.size(0)
    return total_loss / len(loader.dataset)

In [25]:
# 실행 예시
if __name__ == "__main__":
    import numpy as np

    # (1) 데이터 로딩—예: NumPy array of shape [T, C]
    data = numeric_df                             # 내 데이터
    # 2) 정규화
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)
    
    seq_len, pred_len = 4096, 24
    ds = SlidingWindowDataset(data, seq_len, pred_len, stride=24)
    train_size = int(len(ds)*0.8)
    val_size   = len(ds) - train_size
    train_ds, val_ds = torch.utils.data.random_split(ds, [train_size, val_size])
    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=8)

    # (2) 모델/최적화/손실함수
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CNN_LSTM(in_channels=data.shape[1]).to(device)
    optim = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    # (3) 학습 루프
    epochs = 200
    for e in range(1, epochs+1):
        tr_loss = train(model, train_loader, optim, criterion, device)
        val_loss = evaluate(model, val_loader,   criterion, device)
        print(f"Epoch {e:02d} | Train Loss: {tr_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    # (4) 예측
    #x_sample, _ = ds[0]
    #x_sample = x_sample.unsqueeze(0).to(device)  # [1, C, seq_len]
    #y_pred = model(x_sample)                     # [1, pred_len]
    #print("Next 24h forecast:", y_pred.cpu().numpy().flatten())

Epoch 01 | Train Loss: 0.0631 | Val Loss: 0.0218
Epoch 02 | Train Loss: 0.0197 | Val Loss: 0.0180
Epoch 03 | Train Loss: 0.0185 | Val Loss: 0.0170
Epoch 04 | Train Loss: 0.0172 | Val Loss: 0.0167
Epoch 05 | Train Loss: 0.0170 | Val Loss: 0.0170
Epoch 06 | Train Loss: 0.0168 | Val Loss: 0.0171
Epoch 07 | Train Loss: 0.0162 | Val Loss: 0.0165
Epoch 08 | Train Loss: 0.0164 | Val Loss: 0.0166
Epoch 09 | Train Loss: 0.0159 | Val Loss: 0.0164
Epoch 10 | Train Loss: 0.0157 | Val Loss: 0.0202
Epoch 11 | Train Loss: 0.0169 | Val Loss: 0.0183
Epoch 12 | Train Loss: 0.0163 | Val Loss: 0.0184
Epoch 13 | Train Loss: 0.0150 | Val Loss: 0.0180
Epoch 14 | Train Loss: 0.0154 | Val Loss: 0.0171
Epoch 15 | Train Loss: 0.0147 | Val Loss: 0.0176
Epoch 16 | Train Loss: 0.0147 | Val Loss: 0.0170
Epoch 17 | Train Loss: 0.0149 | Val Loss: 0.0172
Epoch 18 | Train Loss: 0.0152 | Val Loss: 0.0185
Epoch 19 | Train Loss: 0.0142 | Val Loss: 0.0177
Epoch 20 | Train Loss: 0.0139 | Val Loss: 0.0176
Epoch 21 | Train Los

In [26]:
    # 6) 샘플 예측
    x_sample, y_true = ds[0]
    x_sample = x_sample.unsqueeze(0).to(device)  # [1, C, seq_len]
    y_pred_scaled = model(x_sample).cpu().detach().numpy().flatten()
    y_true_scaled = y_true.numpy()

In [27]:
# --- 역스케일링 시작 ---
import numpy as np
solar_idx = 0

# 전체 피처 개수
C = data.shape[1]

# 예측/실제 배열을 (pred_len, C) 모양으로 만들고 
pred_full = np.zeros((pred_len, C), dtype=np.float32)
true_full = np.zeros((pred_len, C), dtype=np.float32)

# solar generation 채널(solar_idx)에만 값 채우기
pred_full[:, solar_idx] = y_pred_scaled
true_full[:, solar_idx] = y_true_scaled

# MinMaxScaler.inverse_transform
pred_orig = scaler.inverse_transform(pred_full)[:, solar_idx]
true_orig = scaler.inverse_transform(true_full)[:, solar_idx]
# --- 역스케일링 끝 ---

print("True next-24h (원단위):", true_orig)
print("Pred next-24h (원단위):", pred_orig)

True next-24h (원단위): [ 6.0787883   3.021766    1.319169    0.14787701  0.05296     0.05288
  0.0448      0.          0.          0.          0.          0.
  0.          0.063022    1.079396    2.910719    5.464043    8.924349
 10.057772   10.173965    9.227369   10.90645    10.762956    9.49266   ]
Pred next-24h (원단위): [ 6.2275887   3.3489795   1.5331137   0.18458556  0.2249216   0.11592311
  0.22410633  0.07984954 -0.01098471 -0.0453523  -0.05625914 -0.11521157
 -0.01218295  0.14496407  0.9476556   2.8665648   5.8001394   8.711964
  9.398187    9.538769    9.32058    10.021412   10.532715    9.200582  ]


In [30]:
# --- RMSE 계산 ---
import numpy as np

rmse = np.sqrt(np.mean((pred_orig - true_orig) ** 2))
print(f"RMSE: {rmse:.4f}")

RMSE: 0.3041
MAPE: 6001800721203200.00%
R2: 0.9950


## 보름치 데이터
보름치 데이터로 하루 예측한 RMSE와
디퓨전 생성모형을 통해 보름치 데이터를 여러개 만들어 넣은 걸 합친 데이터로 하루 예측한 RMSE로 

두 가지 경우를 비교

In [27]:
# 
df = pd.read_csv('merged_data_processed_seoul.csv', low_memory=False)

seq_len  = 720   # 학습용: 보름치 (1h 단위 → 360h)
pred_len =  24   # 테스트용: 1일치 (24h)

# 꼬리(tail)에서 잘라내기
#    – train_data: 마지막(pred_len)시간 바로 앞의 seq_len시간
#    – test_data : 마지막 pred_len시간
df_short = df[-(seq_len + pred_len) : ]  # 
numeric_df_short = df_short.drop(columns=['Idx','date','time','일시'])
numeric_df_short = numeric_df_short.apply(pd.to_numeric, errors='coerce')

In [19]:
import torch, numpy as np, random, gc

NUM_RUNS = 100
RMSEs    = []

for run in range(1, NUM_RUNS+1):

    # ── 1) 랜덤시드 고정 (run 값으로 변주) ────────────
    seed = 42 + run
    torch.manual_seed(seed);   np.random.seed(seed);   random.seed(seed)
    torch.cuda.manual_seed_all(seed)

    if __name__ == "__main__":
        import numpy as np

        #  데이터 로딩—예: NumPy array of shape [T, C]
        data = numeric_df_short                             # 내 데이터
        #  정규화
        scaler = MinMaxScaler()
        data = scaler.fit_transform(data)
    
        seq_len, pred_len = 24, 24
        ds = SlidingWindowDataset(data, seq_len, pred_len, stride=24)
        train_size = int(len(ds)*0.8)
        val_size   = len(ds) - train_size
        train_ds, val_ds = torch.utils.data.random_split(ds, [train_size, val_size])
        train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
        val_loader   = DataLoader(val_ds,   batch_size=8)

        # 모델/최적화/손실함수
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = CNN_LSTM(in_channels=data.shape[1]).to(device)
        optim = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.MSELoss()

        # 학습 루프
        epochs = 100
        for e in range(1, epochs+1):
            tr_loss = train(model, train_loader, optim, criterion, device)
            val_loss = evaluate(model, val_loader,   criterion, device)
            #print(f"Epoch {e:02d} | Train Loss: {tr_loss:.4f} | Val Loss: {val_loss:.4f}")
    
        # 샘플 예측
        x_sample, y_true = ds[0]
        x_sample = x_sample.unsqueeze(0).to(device)  # [1, C, seq_len]
        y_pred_scaled = model(x_sample).cpu().detach().numpy().flatten()
        y_true_scaled = y_true.numpy()

    # --- 역스케일링 시작 ---
    solar_idx = 0

    # 전체 피처 개수
    C = data.shape[1]

    # 예측/실제 배열을 (pred_len, C) 모양으로 만들고 
    pred_full = np.zeros((pred_len, C), dtype=np.float32)
    true_full = np.zeros((pred_len, C), dtype=np.float32)

    # solar generation 채널(solar_idx)에만 값 채우기
    pred_full[:, solar_idx] = y_pred_scaled
    true_full[:, solar_idx] = y_true_scaled

    # MinMaxScaler.inverse_transform
    pred_orig = scaler.inverse_transform(pred_full)[:, solar_idx]
    true_orig = scaler.inverse_transform(true_full)[:, solar_idx]
    # --- 역스케일링 끝 ---

    # --- RMSE 계산 ---
    import numpy as np

    rmse = np.sqrt(np.mean((pred_orig - true_orig) ** 2))
    #print(f"RMSE: {rmse:.4f}")


    RMSEs.append(rmse)       # 결과 저장
    print(f"[{run:03d}/{NUM_RUNS}] RMSE = {rmse:.4f}")
    
    # ── 3) GPU 메모리·파이썬 객체 청소 (필수는 아니지만 안전) ──
    del model, train_loader, val_loader
    torch.cuda.empty_cache(); gc.collect()

# ── 4) 통계량 출력 ─────────────────────────────────────
RMSEs = np.array(RMSEs)
mean  = RMSEs.mean()
var   = RMSEs.var(ddof=0)          # 불편분산은 ddof=1
std   = RMSEs.std(ddof=0)

print("\n========== 100-run Summary ==========")
print(f"Mean RMSE  : {mean:.4f}")
print(f"Variance   : {var:.6f}")
print(f"Std. Dev.  : {std:.4f}")


[001/100] RMSE = 0.4662
[002/100] RMSE = 0.7495
[003/100] RMSE = 0.7263
[004/100] RMSE = 0.6411
[005/100] RMSE = 1.5999
[006/100] RMSE = 0.5787
[007/100] RMSE = 0.6348
[008/100] RMSE = 0.7705
[009/100] RMSE = 0.6743
[010/100] RMSE = 0.5403
[011/100] RMSE = 0.7780
[012/100] RMSE = 0.6146
[013/100] RMSE = 0.8313
[014/100] RMSE = 0.6395
[015/100] RMSE = 0.6787
[016/100] RMSE = 0.5581
[017/100] RMSE = 0.5352
[018/100] RMSE = 0.4561
[019/100] RMSE = 0.8092
[020/100] RMSE = 1.0141
[021/100] RMSE = 0.5592
[022/100] RMSE = 0.7798
[023/100] RMSE = 0.6662
[024/100] RMSE = 0.6364
[025/100] RMSE = 0.5048
[026/100] RMSE = 0.4885
[027/100] RMSE = 0.8455
[028/100] RMSE = 0.4486
[029/100] RMSE = 0.6206
[030/100] RMSE = 0.5901
[031/100] RMSE = 0.4101
[032/100] RMSE = 0.5626
[033/100] RMSE = 0.6233
[034/100] RMSE = 0.4702
[035/100] RMSE = 1.1726
[036/100] RMSE = 0.6664
[037/100] RMSE = 0.6852
[038/100] RMSE = 0.9758
[039/100] RMSE = 0.5120
[040/100] RMSE = 0.6660
[041/100] RMSE = 0.5082
[042/100] RMSE =

In [33]:
mape

np.float32(inf)

## 보름치 데이터
보름치 데이터로 하루 예측한 RMSE와

디퓨전 생성모형을 통해 보름치 데이터를 여러개 만들어 넣은 걸 합친 데이터로 하루 예측한 RMSE로 

두 가지 경우를 비교

### 보름치 데이터로 하루 예측한 RMSE

### 디퓨전 생성모형을 통해 보름치 데이터를 여러개 만들어 넣은 걸 합친 데이터로 하루 예측한 RMSE

In [31]:
# 
df = pd.read_csv('merged_data_processed_seoul.csv', low_memory=False)

seq_len  = 720   # 학습용: 보름치 (1h 단위 → 360h)
pred_len =  24   # 테스트용: 1일치 (24h)

# 꼬리(tail)에서 잘라내기
#    – train_data: 마지막(pred_len)시간 바로 앞의 seq_len시간
#    – test_data : 마지막 pred_len시간
df_short = df[-(seq_len + pred_len) : ]  # 
numeric_df_short = df_short.drop(columns=['Idx','date','time','일시'])
numeric_df_short = numeric_df_short.apply(pd.to_numeric, errors='coerce')

In [44]:
import pickle

# 'array.pkl'에 저장된 NumPy 배열을 불러오기
with open('/home1/gkrtod35/Diffusion-TS/merged_results/merged_seq720.pkl', 'rb') as f:
    merged = pickle.load(f)

In [45]:
# 합성 데이터에서 X_synth, Y_synth 뽑기
X_synth = merged[:, :seq_len, :]            # (N_synth, seq_len, C)
Y_synth = merged[:, seq_len:seq_len+pred_len, 0]  # (N_synth, pred_len)  # 채널0=타깃

In [46]:
#  데이터 로딩—예: NumPy array of shape [T, C]
data = numeric_df_short                             # 내 데이터
        #  정규화
scaler = MinMaxScaler()
data = scaler.fit_transform(data)
    
seq_len, pred_len = 24, 24

# data.shape == (T, C)
ds_real = SlidingWindowDataset(data, seq_len=seq_len, pred_len=pred_len, stride=24)
# ds_real.X.shape == (N_real, seq_len, C)
# ds_real.Y.shape == (N_real, pred_len)

X_real = ds_real.X.numpy()
Y_real = ds_real.Y.numpy()

In [49]:
X_real.shape

(1200, 48, 19)

In [50]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# 1) 합치기
X_all = np.concatenate([X_real, X_synth], axis=0)  # (N_real+N_synth, seq_len, C)
Y_all = np.concatenate([Y_real, Y_synth], axis=0)  # (N_real+N_synth, pred_len)

# 2) 텐서로 변환하고, 채널 축을 앞쪽으로 옮기기: [N, C, seq_len]
X_all = torch.tensor(X_all, dtype=torch.float32).transpose(1,2)
Y_all = torch.tensor(Y_all, dtype=torch.float32)

# 3) Dataset & DataLoader
combined_ds = TensorDataset(X_all, Y_all)


In [ ]:
# ─────────────────────────────────────────────────────────────
# 0) 필요 모듈
# ─────────────────────────────────────────────────────────────
import torch, numpy as np, random, gc
from torch.utils.data import DataLoader, random_split
from tqdm.auto import tqdm                    # 진행-bar 보기 좋게
# 이미 선언돼 있는 클래스·함수·데이터:
#   combined_ds, data, scaler, pred_len
#   CNN_LSTM_Transformer, train_epoch, eval_epoch

# ─────────────────────────────────────────────────────────────
# 1) 단일 실험(학습→RMSE) 함수
# ─────────────────────────────────────────────────────────────
def run_once(run_idx: int, epochs: int = 100, batch: int = 8) -> float:
    """학습 1회 수행 후 RMSE 반환"""
    # 1-a. 시드 고정 (run_idx 로 변주)
    seed = 42 + run_idx
    torch.manual_seed(seed); np.random.seed(seed); random.seed(seed)
    torch.cuda.manual_seed_all(seed)

    # 1-b. train/val 재분할
    n_train = int(len(combined_ds) * 0.8)
    n_val   = len(combined_ds) - n_train
    train_ds, val_ds = random_split(combined_ds, [n_train, n_val])

    train_loader = DataLoader(train_ds, batch_size=batch, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=batch)

    # 1-c. 모델·옵티마이저·손실
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model  = CNN_LSTM_Transformer(in_channels=data.shape[1]).to(device)
    optim  = torch.optim.Adam(model.parameters(), lr=1e-3)
    crit   = torch.nn.MSELoss()

    # 1-d. 학습 루프
    for _ in range(epochs):
        train_epoch(model, train_loader, optim, crit, device)
        # val_loss = eval_epoch(model, val_loader, crit, device)  # 선택

    # 1-e. 샘플 예측 (첫 윈도우)
    x_sample, y_true = combined_ds[0]
    x_sample = x_sample.unsqueeze(0).to(device)
    y_pred_s = model(x_sample).cpu().detach().numpy().flatten()
    y_true_s = y_true.numpy()

    # 1-f. 역스케일링 & RMSE
    solar_idx = 0
    C = data.shape[1]
    pf, tf = np.zeros((pred_len, C), np.float32), np.zeros((pred_len, C), np.float32)
    pf[:, solar_idx] = y_pred_s; tf[:, solar_idx] = y_true_s
    pred = scaler.inverse_transform(pf)[:, solar_idx]
    true = scaler.inverse_transform(tf)[:, solar_idx]
    rmse = np.sqrt(np.mean((pred - true) ** 2))

    # 1-g. 메모리 정리
    del model, train_loader, val_loader; torch.cuda.empty_cache(); gc.collect()
    return rmse

# ─────────────────────────────────────────────────────────────
# 2) 100-run 반복
# ─────────────────────────────────────────────────────────────
NUM_RUNS = 10
rmse_list = []

for i in tqdm(range(NUM_RUNS), desc="100 runs"):
    rmse_val = run_once(i+1)
    rmse_list.append(rmse_val)

rmse_arr = np.array(rmse_list)
mean_rmse = rmse_arr.mean()
std_rmse  = rmse_arr.std(ddof=0)      # 표본표준편차는 ddof=1

print("\n========== 100-run Summary ==========")
print(f"Mean RMSE : {mean_rmse:.4f}")
print(f"Std  RMSE : {std_rmse:.4f}")

In [44]:
from torch.utils.data import random_split

# 5) train/val 
if __name__ == "__main__":
    n_train = int(len(combined_ds) * 0.8)
    n_val   = len(combined_ds) - n_train
    train_ds, val_ds = random_split(combined_ds, [n_train, n_val])

    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=8)

    # 6) 모델·옵티마이저·손실함수 세팅
    device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CNN_LSTM(in_channels=data.shape[1]).to(device)
    optim     = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    # 학습 루프
    epochs = 30
    for e in range(1, epochs+1):
        tr_loss = train(model, train_loader, optim, criterion, device)
        val_loss = evaluate(model, val_loader,   criterion, device)
        print(f"Epoch {e:02d} | Train Loss: {tr_loss:.4f} | Val Loss: {val_loss:.4f}")

    # 8) 샘플 예측 + 역스케일링 + RMSE 계산
    x_sample, y_true = combined_ds[0]
    x_sample = x_sample.unsqueeze(0).to(device)  # [1, C, seq_len]
    y_pred_scaled = model(x_sample).cpu().detach().numpy().flatten()
    y_true_scaled = y_true.numpy()

    # 역스케일링
    solar_idx = 0
    C = data.shape[1]
    pred_full = np.zeros((pred_len, C), dtype=np.float32)
    true_full = np.zeros((pred_len, C), dtype=np.float32)
    pred_full[:, solar_idx] = y_pred_scaled
    true_full[:, solar_idx] = y_true_scaled
    pred_orig = scaler.inverse_transform(pred_full)[:, solar_idx]
    true_orig = scaler.inverse_transform(true_full)[:, solar_idx]

    rmse = np.sqrt(np.mean((pred_orig - true_orig)**2))
    print(f"RMSE: {rmse:.4f}")

Epoch 01 | Train Loss: 7.4492 | Val Loss: 4.8954
Epoch 02 | Train Loss: 3.2717 | Val Loss: 2.8391
Epoch 03 | Train Loss: 2.6419 | Val Loss: 2.6577
Epoch 04 | Train Loss: 2.5741 | Val Loss: 2.5186
Epoch 05 | Train Loss: 2.2193 | Val Loss: 2.0384
Epoch 06 | Train Loss: 1.4519 | Val Loss: 0.8760
Epoch 07 | Train Loss: 0.6861 | Val Loss: 0.4759
Epoch 08 | Train Loss: 0.3145 | Val Loss: 0.3119
Epoch 09 | Train Loss: 0.1906 | Val Loss: 0.2356
Epoch 10 | Train Loss: 0.1447 | Val Loss: 0.2216
Epoch 11 | Train Loss: 0.1232 | Val Loss: 0.2195
Epoch 12 | Train Loss: 0.1204 | Val Loss: 0.2121
Epoch 13 | Train Loss: 0.1137 | Val Loss: 0.2135
Epoch 14 | Train Loss: 0.1123 | Val Loss: 0.2118
Epoch 15 | Train Loss: 0.1120 | Val Loss: 0.2139
Epoch 16 | Train Loss: 0.1101 | Val Loss: 0.2136
Epoch 17 | Train Loss: 0.1090 | Val Loss: 0.2116
Epoch 18 | Train Loss: 0.1108 | Val Loss: 0.2107
Epoch 19 | Train Loss: 0.1101 | Val Loss: 0.2119
Epoch 20 | Train Loss: 0.1090 | Val Loss: 0.2125
Epoch 21 | Train Los